# CrewAI Examples: Building Teams of AI Agents

This notebook provides hands-on examples of using CrewAI for building collaborative AI agent teams. You can run these examples to gain practical experience with this role-based framework.

## Useful Resources

- [Official CrewAI Documentation](https://docs.crewai.com/)
- [CrewAI GitHub Repository](https://github.com/joaomdmoura/crewAI)
- [CrewAI Examples Repository](https://github.com/joaomdmoura/crewAI-examples)
- [CrewAI Community Discord](https://discord.gg/X4JWnZnX)

## Setup Instructions

First, let's install the necessary packages:

In [ ]:
# Install required packages
!pip install crewai langchain-openai

# For web search capabilities
# !pip install crewai[serper]

## Setting API Keys

For these examples to work, you'll need to set your API keys for OpenAI and other services. Replace the placeholder with your actual API keys.

In [ ]:
import os

# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # Replace with your actual API key

# If you're using Azure OpenAI
# os.environ["AZURE_OPENAI_API_KEY"] = "your-azure-api-key-here"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource-name.openai.azure.com/"

# If you're using the Serper search tool
# os.environ["SERPER_API_KEY"] = "your-serper-key-here"

## Example 1: Basic CrewAI Team with OpenAI

Let's start with a simple two-agent crew using OpenAI's model to demonstrate the core concepts of CrewAI.

In [ ]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

# Create specialized agents
researcher = Agent(
    role="Research Analyst",
    goal="Find comprehensive information on AI frameworks",
    backstory="You are an expert researcher with 15 years of experience in AI technology analysis. You're known for your thorough and unbiased technical evaluations.",
    llm=llm,
    verbose=True
)

writer = Agent(
    role="Technical Writer",
    goal="Create clear, engaging content about technical subjects",
    backstory="You are a skilled technical writer who can explain complex concepts in accessible language. You have a talent for creating compelling narratives around technical topics.",
    llm=llm,
    verbose=True
)

# Define tasks for each agent
research_task = Task(
    description="Research the top 3 AI agent frameworks (LangChain, AutoGen, and CrewAI). Compare their architectures, strengths, and limitations.",
    agent=researcher
)

writing_task = Task(
    description="Using the research provided, create a comprehensive guide to AI agent frameworks. The content should be informative, engaging, and accessible to developers with basic AI knowledge.",
    agent=writer
)

# Form a crew with these agents and tasks
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    verbose=2  # Increase verbosity to see interactions
)

# Execute the crew's workflow
result = crew.kickoff()
print("\nFinal Result:\n")
print(result)

## Example 2: Task Dependencies

Now let's explore how to create tasks that depend on the output of previous tasks.

In [ ]:
from crewai import Agent, Task, Crew, Process

# Create agents for a content production team
researcher = Agent(
    role="Research Analyst",
    goal="Find accurate and comprehensive information on assigned topics",
    backstory="You are a meticulous researcher with a talent for finding reliable information from diverse sources.",
    llm=llm
)

writer = Agent(
    role="Content Writer",
    goal="Create engaging and informative content based on research",
    backstory="You are a talented writer who can transform complex research into compelling narratives.",
    llm=llm
)

editor = Agent(
    role="Content Editor",
    goal="Ensure content is polished, accurate, and optimized for engagement",
    backstory="You have a keen eye for detail and a deep understanding of what makes content successful.",
    llm=llm
)

# Create tasks with dependencies
research_task = Task(
    description="Research the latest advancements in artificial intelligence for beginners. Focus on practical applications and simple explanations.",
    agent=researcher
)

writing_task = Task(
    description="Write a 'Beginner's Guide to AI' article based on the provided research. Make it engaging and accessible to non-technical readers.",
    agent=writer,
    context=[research_task]  # This task depends on the research task
)

editing_task = Task(
    description="Edit the article for clarity, engagement, and accuracy. Ensure it's approachable for beginners while remaining technically accurate.",
    agent=editor,
    context=[writing_task]  # This task depends on the writing task
)

# Create a crew with sequential process
crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research_task, writing_task, editing_task],
    process=Process.sequential,  # Tasks will run in the defined order
    verbose=1
)

# Execute the workflow
result = crew.kickoff()
print("\nFinal Edited Article:\n")
print(result)

## Example 3: Adding Tools to Agents

Let's create agents with tools that allow them to interact with external systems.

Note: This example requires a Serper API key for web search functionality. You'll need to uncomment and set the API key above.

In [ ]:
# Uncomment this code if you have set up your Serper API key
"""
from crewai import Agent, Task, Crew
from crewai.tools import SerperDevTool, WebsiteSearchTool

# Initialize tools
search_tool = SerperDevTool()
website_tool = WebsiteSearchTool()

# Create a researcher agent with tools
researcher = Agent(
    role="Technology Researcher",
    goal="Find the latest and most accurate information on technology trends",
    backstory="You are a technology analyst who stays current on the latest developments in tech. You use a variety of online sources to inform your analysis.",
    tools=[search_tool, website_tool],
    llm=llm,
    verbose=True
)

# Create an analyst agent
analyst = Agent(
    role="Technology Analyst",
    goal="Analyze technology trends and provide strategic insights",
    backstory="You are an expert at interpreting technology trends and making predictions about future developments.",
    llm=llm,
    verbose=True
)

# Define research task with tool use
research_task = Task(
    description="Research the current state and future trends of AI agent frameworks. Use search tools to find the latest information online.",
    agent=researcher
)

# Define analysis task
analysis_task = Task(
    description="Based on the research, analyze the potential impact of these AI agent frameworks on software development over the next 2 years. Identify key opportunities and challenges.",
    agent=analyst,
    context=[research_task]
)

# Form a crew
crew = Crew(
    agents=[researcher, analyst],
    tasks=[research_task, analysis_task],
    verbose=2
)

# Execute the workflow
result = crew.kickoff()
print("\nFinal Analysis:\n")
print(result)
"""

## Example 4: Custom Tool Creation

Let's create a custom tool that our agents can use.

In [ ]:
from crewai import Agent, Task, Crew
from langchain.tools import BaseTool
import datetime

# Create a custom calendar tool
class CalendarTool(BaseTool):
    name = "Calendar"
    description = "Useful for checking the current date and upcoming events"
    
    def _run(self, query: str) -> str:
        current_date = datetime.datetime.now()
        
        # Mock calendar data
        events = {
            "2023-06-15": ["AI Conference", "Team Meeting"],
            "2023-06-20": ["Product Launch", "Marketing Review"],
            "2023-06-25": ["Quarterly Planning"],
            "2023-07-01": ["New Fiscal Year", "Budget Review"],
        }
        
        if "date" in query.lower() or "today" in query.lower():
            return f"Today's date is {current_date.strftime('%Y-%m-%d')}"
        elif "event" in query.lower() or "schedule" in query.lower():
            response = "Upcoming events:\n"
            for date, event_list in events.items():
                response += f"- {date}: {', '.join(event_list)}\n"
            return response
        else:
            return "I can provide information about today's date or upcoming events. Please specify what you're looking for."
    
    async def _arun(self, query: str) -> str:
        return self._run(query)

# Create a calculator tool
class CalculatorTool(BaseTool):
    name = "Calculator"
    description = "Useful for performing mathematical calculations"
    
    def _run(self, query: str) -> str:
        try:
            return str(eval(query))
        except Exception as e:
            return f"Error in calculation: {str(e)}"
    
    async def _arun(self, query: str) -> str:
        return self._run(query)

# Initialize tools
calendar_tool = CalendarTool()
calculator_tool = CalculatorTool()

# Create an assistant agent with tools
assistant = Agent(
    role="Executive Assistant",
    goal="Help plan and organize the executive's schedule and tasks",
    backstory="You are an efficient executive assistant who helps busy professionals stay organized and prepared.",
    tools=[calendar_tool, calculator_tool],
    llm=llm,
    verbose=True
)

# Define a planning task
planning_task = Task(
    description="Create a schedule for the upcoming product launch. Include key dates, calculate required resources, and identify potential scheduling conflicts.",
    agent=assistant
)

# Form a crew
crew = Crew(
    agents=[assistant],
    tasks=[planning_task],
    verbose=2
)

# Execute the workflow
result = crew.kickoff()
print("\nPlanning Result:\n")
print(result)

## Example 5: Hierarchical Team Structure

Let's create a more complex team structure with a manager overseeing other agents.

In [ ]:
from crewai import Agent, Task, Crew

# Create specialized agents
manager = Agent(
    role="Project Manager",
    goal="Oversee the creation of a comprehensive market analysis report",
    backstory="You are an experienced project manager who excels at coordinating teams and ensuring high-quality deliverables.",
    llm=llm
)

researcher1 = Agent(
    role="Market Researcher",
    goal="Gather comprehensive market data",
    backstory="You are a specialist in gathering and analyzing market data from various industries.",
    llm=llm
)

researcher2 = Agent(
    role="Competitor Analyst",
    goal="Analyze competitor strategies and positioning",
    backstory="You are an expert at analyzing competitors and identifying their strengths and weaknesses.",
    llm=llm
)

writer = Agent(
    role="Report Writer",
    goal="Create a compelling and insightful market analysis report",
    backstory="You are skilled at synthesizing complex information into clear, actionable reports.",
    llm=llm
)

# Manager tasks
planning_task = Task(
    description="Create a detailed plan for the market analysis project, including key areas to research and analysis approaches.",
    agent=manager
)

final_review_task = Task(
    description="Review all components of the analysis and compile them into a cohesive final report with key insights and recommendations.",
    agent=manager,
    context=["market_research_task", "competitor_analysis_task", "report_writing_task"]
)

# Researcher tasks
market_research_task = Task(
    description="Research the current state of the AI agent framework market. Include market size, growth trends, and key customer segments.",
    agent=researcher1,
    context=[planning_task]
)

competitor_analysis_task = Task(
    description="Analyze the top 3 companies in the AI agent framework space. Identify their strengths, weaknesses, and unique selling propositions.",
    agent=researcher2,
    context=[planning_task]
)

# Writer task
report_writing_task = Task(
    description="Write a comprehensive market analysis report based on the market research and competitor analysis. The report should be insightful and actionable.",
    agent=writer,
    context=[market_research_task, competitor_analysis_task]
)

# Form a crew with a hierarchical structure
crew = Crew(
    agents=[manager, researcher1, researcher2, writer],
    tasks=[planning_task, market_research_task, competitor_analysis_task, report_writing_task, final_review_task],
    process=Process.sequential,
    verbose=1
)

# Execute the workflow
result = crew.kickoff()
print("\nFinal Market Analysis Report:\n")
print(result)

## Further Exploration

These examples demonstrate the basics of CrewAI's role-based approach to agent teams. For more advanced usage, consider exploring:

1. **Agent Delegation**: Creating agents that can delegate subtasks to other agents
2. **Custom Processes**: Implementing custom workflow processes beyond sequential and hierarchical
3. **Integration with External Systems**: Using agents to interface with databases, APIs, etc.
4. **Human-in-the-Loop Workflows**: Incorporating human feedback into agent processes
5. **Long-Running Operations**: Managing state for complex, multi-day workflows

Check out the [CrewAI documentation](https://docs.crewai.com/) for more examples and advanced usage scenarios.

## Example 6: Using Azure OpenAI

This example demonstrates how to use CrewAI with Azure OpenAI instead of regular OpenAI.

In [ ]:
from crewai import Agent, Task, Crew
from langchain_openai import AzureChatOpenAI

# Initialize the Azure OpenAI LLM
azure_llm = AzureChatOpenAI(
    azure_deployment="gpt-4",  # The deployment name you chose when you deployed the GPT-4 model
    openai_api_version="2023-05-15",
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://your-resource-name.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY", "your-azure-openai-api-key"),
    temperature=0.7
)

# Create specialized agents
researcher = Agent(
    role="Research Analyst",
    goal="Find comprehensive information on AI frameworks",
    backstory="You are an expert researcher with 15 years of experience in AI technology analysis. You're known for your thorough and unbiased technical evaluations.",
    llm=azure_llm,
    verbose=True
)

writer = Agent(
    role="Technical Writer",
    goal="Create clear, engaging content about technical subjects",
    backstory="You are a skilled technical writer who can explain complex concepts in accessible language. You have a talent for creating compelling narratives around technical topics.",
    llm=azure_llm,
    verbose=True
)

# Define tasks for each agent
research_task = Task(
    description="Research the top 3 AI agent frameworks. Compare their architectures, strengths, and limitations.",
    agent=researcher
)

writing_task = Task(
    description="Using the research provided, create a comprehensive guide to AI agent frameworks. The content should be informative, engaging, and accessible to developers with basic AI knowledge.",
    agent=writer,
    context=[research_task]  # This task depends on the research task
)

# Form a crew with these agents and tasks
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    verbose=2  # Increased verbosity to see the process
)

# Execute the crew's workflow
# Uncomment to run (make sure your Azure OpenAI credentials are set correctly)
# result = crew.kickoff()
# print(result)

## Example 7: Using CrewAI Flows

Flows provide more structured automation with granular control over workflow execution.

In [ ]:
from crewai import Flow, Stage
from langchain_openai import ChatOpenAI

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

# Create flow stages
research_stage = Stage(
    name="research",
    instructions="Research AI frameworks comprehensively",
    llm=llm
)

writing_stage = Stage(
    name="writing",
    instructions="Write a detailed article based on the research",
    llm=llm
)

editing_stage = Stage(
    name="editing",
    instructions="Edit the article for clarity, flow, and accuracy",
    llm=llm
)

# Create a flow
content_flow = Flow(
    name="content_creation",
    description="Researching and writing an article about AI frameworks",
    stages=[research_stage, writing_stage, editing_stage]
)

# Execute flow with initial input
# Uncomment to run
# result = content_flow.run(input={"topic": "AI agent frameworks comparison"})
# print(result)

## Conclusion

In this notebook, we've explored various examples of using CrewAI to build teams of collaborative AI agents. We've demonstrated how to:

1. Create basic agent teams with OpenAI and Azure OpenAI
2. Use specialized tools with agents
3. Create complex multi-agent interactions
4. Use CrewAI Flows for more structured automation

CrewAI offers a powerful framework for orchestrating role-based autonomous AI agents, making it easier to build systems where multiple agents collaborate in human-like teams to solve complex tasks.